In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

# 1. 데이터 전처리

**X_train, X_test를 합쳐서 전처리 작업을 할 경우**
- 전처리 작업 후 분리해서 X_train은 학습모델을 생성하는 데에, X_test는 제출용 값을 구하는 데에 사용한다.
- `sclaer.fit_transform(X_train + X_test)`

**X_train, X_test를 각각 전처리 작업을 할 경우**
- X_train에 적용된 전처리가 그대로 X_test에 적용되어야 한다.
- `scaler.fit(X_train)`: 전처리에 필요한 값 준비(scaler를 return)
- `scaler.transform(X_train)`: X_train 전처리 실행
- `scaler.transform(X_test)`: X_test 전처리 실행

## 1-1. 데이터 정규화

- `fit(X_train)`: 전처리에 필요한 값 준비
  - return scaler
- `transform(X_trian)`: 전처리 실행
  - return 변환된 값
- `fit_transform(X_train)`: 전처리에 필요한 값 준비 및 처리
  - return 변환된 값

### ① StandardScaler

- 기존 변수의 범위를 정규분포로 변환하는 것으로 데이터의 최소 최대를 모를 때 사용한다.
- 모든 피쳐의 평균을 0, 분산을 1로 만든다.
- 이상치가 있다면 평균과 표준편차에 영향을 미치기 때문에 이상치가 많다면 사용하지 않는 것이 좋다(이상치가 있다면 균형 잡힌 척도를 보장할 수 없다).
- 회귀 모델보다 **분류 모델**에 적합하다.

### ② MinMaxScaler

- 데이터의 값들을 0~1 사이의 값으로 변환시킨다.
- 각 변수가 정규분포가 아니거나 표준편차가 작을 때 효과적이다.
- 하지만 StandardSclaer와 같이 이상치 존재에 민감하다.
- 분류 모델보다 **회귀 모델**에 적합하다.
- 보통 이미지 쪽에서 쓰인다.

### ③ MaxAbsScaler

- 절댓값이 0~1 사이에 매핑되도록 한다.
- 즉 0을 기준으로 절댓값이 가장 큰 수가 1 또는 -1 사이로 재조정한다.
- 양수 데이터로만 구성된 특정 데이터셋에서는 MinMaxScaler와 유사하게 동작하며, 큰 이상치에 민감할 수 있다.

### ④ RobustScaler

- 모든 피쳐가 같은 크기를 갖는다는 점이 StandardScaler와 유사하다.
- 평균과 분산이 아닌 중위수(median)과 IQR(사분위수)를 사용한다.
- StandardScaler에 비해 이상치의 영향이 적다.

### ⑤ Normalizer

- 앞선 4가지 방법은 열(columns)을 대상으로 각 피쳐의 통계치를 이용한다.
- 그러나 Normalizser의 경우 각 행(row)마다 정규화가 진행된다.
- 각 변수의 값을 원점으로부터 1만큼 떨어져 있는 범위 내로(유클리드 거리가 1이 되도록) 변환한다.
- 빠르게 학습할 수 있고, 과대적합 롹률을 낮출 수 있다.

Scaling 과정을 마친 후에는 항상 데이터의 describe()를 확인하는 습관이 필요하다.<br>
각 Scaler의 범위 안에 들어오는지 확인해야 한다.

In [2]:
!git clone https://github.com/Soyoung-Yoon/bigdata

fatal: destination path 'bigdata' already exists and is not an empty directory.


In [3]:
df = pd.read_csv('bigdata/iris_data.csv')

In [4]:
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [5]:
# X, Y 데이터로 분리
X = df.iloc[:, :-1]
Y = df.iloc[:, -1] # df['target']

In [6]:
scaledX = StandardScaler().fit_transform(X)
print(scaledX.mean(), scaledX.std())

-4.855375361027351e-16 1.0


In [7]:
scaler = StandardScaler().fit(X)
scaledX = scaler.transform(X)
print(scaledX.mean(), scaledX.std())
# scaledX_submission = scaler.traonsform(X_test)

-4.855375361027351e-16 1.0


# 2. 데이터 분할

`sklearn.model_selection.train_test_split`
```python
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size, train_size, random_state, shuffle, stratify)
```
- `shuffle = True`: 데이터셋을 섞을지 말지 결정 (default=True)
- `stratify`: Y의 지정한 데이터 비율을 유지(층화추출), Y가 **범주형**일 때 사용

In [8]:
# train, test 8:2로 나누기
a = list(range(10))
x = train_test_split(a, test_size=0.2)
print(x)

x_train, x_test = x
print(x_train, x_test)

[[7, 9, 1, 3, 8, 6, 4, 0], [2, 5]]
[7, 9, 1, 3, 8, 6, 4, 0] [2, 5]


In [9]:
# 층화추출 이해하기(비율 맞춰 나누기)
a = [1, 1, 2, 3] * 5
x_train, x_test = train_test_split(a, test_size=0.2, stratify=a) # 1, 2, 3 => 2: 1: 1
print(x_train, x_test)

x_train, x_test = train_test_split(a, test_size=0.4, stratify=a) # 1, 2, 3 => 2: 1: 1
print(x_train, x_test)

[1, 3, 1, 1, 1, 2, 1, 2, 3, 2, 3, 1, 1, 1, 3, 2] [3, 2, 1, 1]
[2, 1, 3, 1, 1, 3, 1, 1, 2, 3, 1, 2] [3, 1, 1, 1, 3, 1, 2, 2]


In [10]:
# 여러 개의 데이터셋을 분할하기 위해서는 행의 개수가 같아야 함
a = list(range(10))
b = list(range(10))

train_test_split(a, b, test_size=0.2, random_state=42)

[[5, 0, 7, 2, 9, 4, 3, 6], [8, 1], [5, 0, 7, 2, 9, 4, 3, 6], [8, 1]]

In [11]:
x_train, x_test, y_train, y_test = train_test_split(scaledX, Y,
                                                    test_size=0.2,
                                                    random_state=42,
                                                    stratify=Y) # 회귀에서는 사용하지 않음
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(120, 4) (30, 4) (120,) (30,)


# 3. 머신러닝 모델(Estimator) Interface

- `fit(X_train, y_train)`: 모델 학습 훈련
- `score(X_test, y_test)`: 성능 측정(1에 가까울수록 좋은 성능)
- `predict(X_test)`: 예측값 반환

In [12]:
help(KNeighborsClassifier)

Help on class KNeighborsClassifier in module sklearn.neighbors._classification:

class KNeighborsClassifier(sklearn.neighbors._base.KNeighborsMixin, sklearn.base.ClassifierMixin, sklearn.neighbors._base.NeighborsBase)
 |  KNeighborsClassifier(n_neighbors=5, *, weights='uniform', algorithm='auto', leaf_size=30, p=2, metric='minkowski', metric_params=None, n_jobs=None)
 |  
 |  Classifier implementing the k-nearest neighbors vote.
 |  
 |  Read more in the :ref:`User Guide <classification>`.
 |  
 |  Parameters
 |  ----------
 |  n_neighbors : int, default=5
 |      Number of neighbors to use by default for :meth:`kneighbors` queries.
 |  
 |  weights : {'uniform', 'distance'}, callable or None, default='uniform'
 |      Weight function used in prediction.  Possible values:
 |  
 |      - 'uniform' : uniform weights.  All points in each neighborhood
 |        are weighted equally.
 |      - 'distance' : weight points by the inverse of their distance.
 |        in this case, closer neighb

```
KNeighborsClassifier(n_neighbors=5, *, weights='uniform', algorithm='auto',
                     leaf_size=30, p=2, metric='minkowski', metric_params=None, n_jobs=None)
```

In [13]:
# k가 3인 knn 분류기 생성
knn = KNeighborsClassifier(n_neighbors=3)

# knn에 train set 전달하여 모델 생성(학습)
knn.fit(x_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [14]:
# 위와 동일한 결과
knn = KNeighborsClassifier(n_neighbors=3).fit(x_train, y_train)

In [15]:
# 평가하기
print('train 성능:', knn.score(x_train, y_train))
print('test 성능:', knn.score(x_test, y_test))

train 성능: 0.9666666666666667
test 성능: 0.9333333333333333


성능이 낮은 경우에는
1. 하이퍼파라미터를 조절하거나,
2. model을 변경하거나,
3. 데이터전처리 변경 등을 통해 성능을 향상할 수 있다.

In [16]:
# knn 모델에 test set 적용하여 예측치 얻기
pred = knn.predict(x_test)
pred

array([0, 2, 1, 1, 0, 1, 0, 0, 2, 1, 2, 2, 2, 1, 0, 0, 0, 1, 1, 1, 0, 2,
       1, 1, 2, 1, 1, 0, 2, 0], dtype=int64)

In [17]:
print(pred)
print(y_test.to_numpy())

[0 2 1 1 0 1 0 0 2 1 2 2 2 1 0 0 0 1 1 1 0 2 1 1 2 1 1 0 2 0]
[0 2 1 1 0 1 0 0 2 1 2 2 2 1 0 0 0 1 1 2 0 2 1 2 2 1 1 0 2 0]


In [18]:
# 예측 확률값
pred_proba = knn.predict_proba(x_test)
print(pred_proba)

[[1.         0.         0.        ]
 [0.         0.         1.        ]
 [0.         1.         0.        ]
 [0.         1.         0.        ]
 [1.         0.         0.        ]
 [0.         0.66666667 0.33333333]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [0.         0.         1.        ]
 [0.         1.         0.        ]
 [0.         0.         1.        ]
 [0.         0.         1.        ]
 [0.         0.         1.        ]
 [0.         1.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [0.         1.         0.        ]
 [0.         1.         0.        ]
 [0.         0.66666667 0.33333333]
 [1.         0.         0.        ]
 [0.         0.         1.        ]
 [0.         1.         0.        ]
 [0.         0.66666667 0.33333333]
 [0.         0.         1.        ]
 [0.         0.66666667 0.33333333]
 [0.         1.         0.        ]
 [1.         0.         0.  

# 4. GridSearchCV 사용

`sklearn.model_selection.GridSearchCV`
```
(estimator, param_grid, , scoring=None, n_jobs=None, refit=True, cv=None, verbose=0, pre_dispatch='2n_jobs', error_score=nan, return_train_score=False)
```
- estimator: 학습 모델
- param_grid: 실행해 볼 하이퍼파라미터 목록, dict 객체
- cv: CrossValidation에 사용할 나누는 개수(default: 5)
- verbose: 0(default): 메시지 출력 안함, 1: 간단한 메시지, 2: 하이퍼파라미터별 메시지 출력
- n_jobs: -1로 지정하면 모든 코어를 다 사용하여 속도가 빨라짐

In [19]:
df['target'].to_numpy()

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int64)

In [20]:
# 데이터 섞기
df2 = df.sample(frac=1, random_state=42)

# X, Y 분리하기
X = df.iloc[:, :-1]
Y = df.iloc[:, -1]

# X에 대해 Scale
scaledX = StandardScaler().fit_transform(X)

# 시도해 볼 하이퍼파라미터 종류 나여하기
params = {'n_neighbors': range(3, 10),
          'p': [1, 2]}

# knn 학습 모델 객체 생성하기
model = KNeighborsClassifier()

# GridSearchCV 객체 만들기 (위에어 생성한 model, params를 사용하고, cv의 개수를 지정함)
gs = GridSearchCV(model, params, cv=5)

# gs를 학습하기
gs.fit(scaledX, Y)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': range(3, 10), 'p': [1, 2]})

`df.sample`
- `n=5`: n개의 row를 random하게 return
- `frac=1`: 전체 row에서 특정 비율의 데이터를 random하게 return

In [21]:
# gs의 주요 attribute 살펴보기
print(dir(gs))

['__abstractmethods__', '__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_check_feature_names', '_check_n_features', '_check_refit_for_multimetric', '_estimator_type', '_format_results', '_get_param_names', '_get_tags', '_more_tags', '_repr_html_', '_repr_html_inner', '_repr_mimebundle_', '_required_parameters', '_run_search', '_select_best_index', '_validate_data', '_validate_params', 'best_estimator_', 'best_index_', 'best_params_', 'best_score_', 'classes_', 'cv', 'cv_results_', 'decision_function', 'error_score', 'estimator', 'fit', 'get_params', 'inverse_transform', 'multimetric_', 'n_features_in_', 'n_jobs', 'n_splits_', 'para

In [22]:
gs.cv_results_

{'mean_fit_time': array([0.00058365, 0.00040021, 0.00085521, 0.00089025, 0.00092573,
        0.00051584, 0.00080414, 0.00120411, 0.00062485, 0.00071182,
        0.00091138, 0.00102663, 0.00101924, 0.00091381]),
 'std_fit_time': array([4.81135710e-04, 4.90154449e-04, 4.40912862e-04, 4.67161941e-04,
        5.03705221e-04, 4.55863945e-04, 4.02101268e-04, 2.47007291e-04,
        5.10625216e-04, 4.03443114e-04, 5.01263722e-04, 1.86619468e-05,
        2.91906023e-04, 1.81191243e-04]),
 'mean_score_time': array([0.00203757, 0.00190377, 0.00177469, 0.00166526, 0.00152822,
        0.00191269, 0.0014442 , 0.00180922, 0.00218315, 0.00218482,
        0.00200953, 0.00209746, 0.00222621, 0.00204067]),
 'std_score_time': array([0.00068809, 0.0004901 , 0.00040644, 0.00053357, 0.00046668,
        0.00045031, 0.00066288, 0.00051603, 0.00052478, 0.00049941,
        0.00043251, 0.00011827, 0.00035987, 0.0006872 ]),
 'param_n_neighbors': masked_array(data=[3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9],
      

In [23]:
result = pd.DataFrame(gs.cv_results_)
result

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_neighbors,param_p,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.000584,0.000481,0.002038,0.000688,3,1,"{'n_neighbors': 3, 'p': 1}",0.966667,0.966667,0.900000,0.933333,1.000000,0.953333,0.033993,6
1,0.000400,0.000490,0.001904,0.000490,3,2,"{'n_neighbors': 3, 'p': 2}",0.966667,0.966667,0.933333,0.900000,1.000000,0.953333,0.033993,6
2,0.000855,0.000441,0.001775,0.000406,4,1,"{'n_neighbors': 4, 'p': 1}",0.966667,0.966667,0.966667,0.900000,0.966667,0.953333,0.026667,6
3,0.000890,0.000467,0.001665,0.000534,4,2,"{'n_neighbors': 4, 'p': 2}",0.966667,0.966667,1.000000,0.866667,0.966667,0.953333,0.045216,6
4,0.000926,0.000504,0.001528,0.000467,5,1,"{'n_neighbors': 5, 'p': 1}",0.966667,0.966667,0.900000,0.900000,1.000000,0.946667,0.040000,13
5,0.000516,0.000456,0.001913,0.000450,5,2,"{'n_neighbors': 5, 'p': 2}",0.966667,0.966667,0.933333,0.933333,1.000000,0.960000,0.024944,3
6,0.000804,0.000402,0.001444,0.000663,6,1,"{'n_neighbors': 6, 'p': 1}",0.966667,0.966667,0.966667,0.866667,1.000000,0.953333,0.045216,6
7,0.001204,0.000247,0.001809,0.000516,6,2,"{'n_neighbors': 6, 'p': 2}",0.966667,0.966667,1.000000,0.900000,1.000000,0.966667,0.036515,1
8,0.000625,0.000511,0.002183,0.000525,7,1,"{'n_neighbors': 7, 'p': 1}",0.966667,0.966667,0.966667,0.900000,1.000000,0.960000,0.032660,3
9,0.000712,0.000403,0.002185,0.000499,7,2,"{'n_neighbors': 7, 'p': 2}",0.966667,0.966667,0.933333,0.900000,1.000000,0.953333,0.033993,6


In [24]:
model = gs.best_estimator_
print(model.score(x_test, y_test), gs.score(x_test, y_test), gs.best_params_)
# refit=True으로 되어있기 때문에 best_estimator로 model에 적용되어 있음
# 따라서 gs.best_estimator_를 적용한 model의 socre와 gs.score(x_test, y_test)는 동일함

0.9666666666666667 0.9666666666666667 {'n_neighbors': 6, 'p': 2}


In [25]:
# 각 class별 예측 확률 구하기 (분류인 경우)
# model.predict(x_test)
model.predict_proba(x_test)

array([[1.        , 0.        , 0.        ],
       [0.        , 0.16666667, 0.83333333],
       [0.        , 1.        , 0.        ],
       [0.        , 1.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [0.        , 0.83333333, 0.16666667],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [0.        , 0.        , 1.        ],
       [0.        , 1.        , 0.        ],
       [0.        , 0.        , 1.        ],
       [0.        , 0.        , 1.        ],
       [0.        , 0.        , 1.        ],
       [0.        , 1.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [0.        , 1.        , 0.        ],
       [0.        , 0.83333333, 0.16666667],
       [0.        , 0.33333333, 0.66666667],
       [1.        , 0.        , 0.        ],
       [0.        , 0.        , 1.        ],
       [0.